In [1]:
import numpy as np
import pandas as pd

from auxiliary import values as v
from auxiliary.data import imaging
from auxiliary.utils.timer import LoadingBar

from nuclei_segmentation.quality_control import metrics, visualization

/home/txete/miniconda3/envs/seg_qc/lib/python3.9/site-packages/timagetk/components/labelled_image.py:42: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
v.data_path = '/mnt/c/Users/ignac/OneDrive/Nacho/CNIC/TFM/Data/'

pred_paths = v.data_path + 'Gr1/Segmentation/Nuclei/QC_CROP/'
gt_path = (
        v.data_path 
        + 'Gr1/Segmentation/Nuclei/QC_CROP/20190208_E2_nuclei_mask_crop_GT.nii.gz'
)

gt = imaging.read_image(gt_path, axes='XYZ')

In [3]:
from itertools import permutations


steps_type = ['2D', '3D']
steps_thr = list(np.round(np.linspace(0.1, .8, 8), 1))
steps_mandatory = ['isotropy', 'normalization']
steps_optional = [
    'bilateral', 
    'anisodiff'
]

steps_permutations = list(permutations(steps_optional))

pipelines_dict = {}

# Create pipelines for each type and threshold
for step in steps_type:
    for thr in steps_thr:
        for i, perm in enumerate(steps_permutations):
            # Create the pipeline with mandatory steps
            pipeline = steps_mandatory + list(perm)

            # Store the pipeline along with the step type and threshold
            pipeline_key = f'pipeline_{step}_{i}_thr_{thr}'
            pipelines_dict[pipeline_key] = {
                'pipeline': pipeline,
                'type': step,
                'threshold': thr
            }

# Print the created pipelines
for name, config in pipelines_dict.items():
    print(f"{name}: {config['pipeline']}, Type: {config['type']}, Threshold: {config['threshold']}")
    
print('Total pipelines:', len(pipelines_dict))

pipeline_2D_0_thr_0.1: ['isotropy', 'normalization', 'bilateral', 'anisodiff'], Type: 2D, Threshold: 0.1
pipeline_2D_1_thr_0.1: ['isotropy', 'normalization', 'anisodiff', 'bilateral'], Type: 2D, Threshold: 0.1
pipeline_2D_0_thr_0.2: ['isotropy', 'normalization', 'bilateral', 'anisodiff'], Type: 2D, Threshold: 0.2
pipeline_2D_1_thr_0.2: ['isotropy', 'normalization', 'anisodiff', 'bilateral'], Type: 2D, Threshold: 0.2
pipeline_2D_0_thr_0.3: ['isotropy', 'normalization', 'bilateral', 'anisodiff'], Type: 2D, Threshold: 0.3
pipeline_2D_1_thr_0.3: ['isotropy', 'normalization', 'anisodiff', 'bilateral'], Type: 2D, Threshold: 0.3
pipeline_2D_0_thr_0.4: ['isotropy', 'normalization', 'bilateral', 'anisodiff'], Type: 2D, Threshold: 0.4
pipeline_2D_1_thr_0.4: ['isotropy', 'normalization', 'anisodiff', 'bilateral'], Type: 2D, Threshold: 0.4
pipeline_2D_0_thr_0.5: ['isotropy', 'normalization', 'bilateral', 'anisodiff'], Type: 2D, Threshold: 0.5
pipeline_2D_1_thr_0.5: ['isotropy', 'normalization', 'a

In [10]:
bar = LoadingBar(len(pipelines_dict))

results = []
for name, config in pipelines_dict.items():
    print(f"\nPipeline: {name}")
    
    file_name = f'20190208_E2_DAPI_decon_0.5_crop_{name}.nii.gz'
    
    out_path = pred_paths + 'Stats/' + file_name.replace('.nii.gz', '.tif')
    pred = imaging.read_image(pred_paths + file_name, axes='XYZ')
    
    dice, _ = metrics.dice_coef(pred, gt, thr_overlap=.6)
    jaccard = metrics.volume_jaccard_index(pred, gt)
    res, stats = metrics.segmentation_stats(pred, gt, thr_overlap=.6)

    results.append({
        'test_name': name,
        'dice': dice,
        'jaccard': jaccard,
        '#cells': stats['total'],
        'correct': stats['correct'],
        'missing': stats['missing'],
        'over_segmented': stats['over_segmented'],
        'under_segmented': stats['under_segmented'],
        'confused': stats['confused']
    })
    
    visualization.save_comparison(pred, res, out_path)
    bar.update()
    
bar.end()


Pipeline: pipeline_2D_0_thr_0.1
INFO     [timagetk.components.labelled_image] Searching the bounding-boxes of 137 labels...
INFO     [timagetk.components.labelled_image] Searching the bounding-boxes of 189 labels...
INFO     [timagetk.components.labelled_image] Searching the bounding-boxes of 137 labels...
INFO     [timagetk.components.labelled_image] Searching the bounding-boxes of 189 labels...
INFO     [timagetk.components.labelled_image] Searching the bounding-boxes of 137 labels...
INFO     [timagetk.components.labelled_image] Searching the bounding-boxes of 189 labels...
[=                                                 ] 3.12%
Pipeline: pipeline_2D_1_thr_0.1
INFO     [timagetk.components.labelled_image] Searching the bounding-boxes of 141 labels...
INFO     [timagetk.components.labelled_image] Searching the bounding-boxes of 189 labels...
INFO     [timagetk.components.labelled_image] Searching the bounding-boxes of 141 labels...
INFO     [timagetk.components.labelled_image] Se

In [6]:
results_df = pd.DataFrame(results)
results_df.to_csv(pred_paths + 'Stats/results.csv')

In [7]:
results_df

,test_name,dice,jaccard,#cells,correct,missing,over_segmented,under_segmented,confused
0,pipeline_2D_0_thr_0.1,0.550007,0.490871,134.0,45.52,2.66,6.72,18.66,29.10
1,pipeline_2D_1_thr_0.1,0.545533,0.488197,137.0,41.61,3.21,10.95,20.44,27.01
2,pipeline_2D_0_thr_0.2,0.549909,0.490736,135.0,44.44,2.66,8.15,18.52,28.89
3,pipeline_2D_1_thr_0.2,0.545533,0.488197,137.0,41.61,3.21,10.95,20.44,27.01
4,pipeline_2D_0_thr_0.3,0.549850,0.490494,137.0,43.07,2.66,9.49,17.52,29.93
5,pipeline_2D_1_thr_0.3,0.548313,0.491230,140.0,40.71,3.21,12.86,20.00,26.43
6,pipeline_2D_0_thr_0.4,0.560717,0.502179,142.0,42.96,2.66,10.56,16.90,29.58
7,pipeline_2D_1_thr_0.4,0.558689,0.500687,147.0,39.46,3.21,15.65,19.05,25.85
8,pipeline_2D_0_thr_0.5,0.607714,0.544054,168.0,39.29,2.66,19.64,15.48,25.60
9,pipeline_2D_1_thr_0.5,0.602157,0.539159,177.0,39.55,3.21,24.29,14.12,22.03
